## **Time series classification using tree-based methods**

#### **Import libraries**

In [ ]:
import numpy as np
import random
import pandas as pd
from sktime.datasets import load_from_tsfile_to_dataframe
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import linkage, fcluster
from collections import Counter
from sklearn.metrics import adjusted_rand_score
from sklearn.preprocessing import MinMaxScaler


#### **Define distance functions**

In [ ]:
def euclidean_distance(x_slice, x_ref):
	x_ref_value = float(np.array(x_ref).flatten()[0])
	return np.sqrt(np.sum((x_slice - x_ref_value) ** 2))

def manhattan_distance(x_slice, x_ref):
	x_ref_value = float(np.array(x_ref).flatten()[0])
	return np.sum(np.abs(x_slice - x_ref_value))

def cosine_distance(x_slice, x_ref):
	"""
	Distanza coseno tra x_ref e x_slice trattata come vettore
	Se vuoi element-wise, considera x_ref broadcasted
	"""
	x_ref_value = float(np.array(x_ref).flatten()[0])
	x_slice_vec = np.array(x_slice)
	# coseno tra vettori: x_ref replicato
	dot = x_slice_vec * x_ref_value
	norm = np.linalg.norm(x_slice_vec) * np.linalg.norm([x_ref_value])
	return 1 - np.sum(dot) / (norm + 1e-9)


#### **Node and PromptTree class**

In [ ]:
class Node():
	def __init__(self, x_barr, channel, bi, ei, df, t):
		self.x_barr = x_barr
		self.channel = channel
		self.begin_idx = bi
		self.end_idx = ei
		self.distance_f_node = df 
		self.threshold = t
		
		# figli
		self.node_dx = None # Node()
		self.node_sx = None # Node()

class LeafNode():
	def __init__(self, class_f):
		self.classification_f = class_f

In [ ]:
class PromptTree():
	def __init__(self, X, y, distance_function):
		# Require
		self.X = X
		self.y = y
		self.current_path = None  # lista di dizionari
		self.distance_functions = self.get_distance_functions()
		self.dist_func_sel = distance_function

		values, counts = np.unique(y, return_counts=True)
		self.most_common_class = values[np.argmax(counts)]  # use when leaf is not able to assign a class

		self.root = self.prompt_tree_fit_algo(self.X, self.y, self.current_path)
		# self.v = [] # vertices
		# self.e = [] # edges
		# self.s = [] # rst
		# self.l = [] # labels

	def prompt_tree_fit_algo(self, X, y, path):
		if path is None:
			path = []
			B = [0]
			E = [0]
		else:
			B, E = [], []
			for node in path:
				B.append(node.begin_idx)
				E.append(node.end_idx)

		# Condizione di stop
		if self.stopping_f(path, X, y):
			if len(X) == 0:  # quando non ha una previsione
				return LeafNode(None)

			leaf_class = self.classification_f(X, y)
			return LeafNode(leaf_class)

		# Candidate intervals
		candidate_intervals = self.promptness_f(X, y, B, np.max(E) + 1)

		# Candidate tests
		candidate_tests = self.sampling_f(X, y, candidate_intervals)

		# Optimal candidate
		optimal_candidate, df_true, df_false = self.optimization_f(X, y, candidate_tests)

		# Recupero valori del nodo
		x_barr, c, begin_idx, end_idx, dist_f, threshold = (
			optimal_candidate[k] for k in ['x_barr', 'channel', 'b', 'e', 'dist_fun', 'threshold']
		)

		# Creazione nodo
		node = Node(x_barr, c, begin_idx, end_idx, dist_f, threshold)

		# Preparazione dati per i rami sinistro e destro
		X_true = pd.DataFrame([x for x, _ in df_true], columns=X.columns)
		y_true = np.array([label for _, label in df_true])
		X_false = pd.DataFrame([x for x, _ in df_false], columns=X.columns)
		y_false = np.array([label for _, label in df_false])

		# Ricorsione ramo sinistro
		path_true = path.copy()
		path_true.append(node)
		node.node_sx = self.prompt_tree_fit_algo(X_true, y_true, path_true)

		# Ricorsione ramo destro
		path_false = path.copy()
		path_false.append(node)
		node.node_dx = self.prompt_tree_fit_algo(X_false, y_false, path_false)

		return node

	def promptness_f(self, X, y, B, max_e):
		''' Propone un set di coppie: canale, intervallo'''
		total_pairs = random.randint(1, len(B)//2 + 1)
		pairs = []

		channels = list(X.columns)
		k = random.randint(1, len(channels))
		selected_channels = random.sample(channels, k)

		for channel in selected_channels:
			for i in range(total_pairs):
				b = random.randrange(B[0], max_e)
				pair = {'channel': channel, "interval": (b, max_e)}
				pairs.append(pair)
		return pairs

	def sampling_f(self, X, y, candidate_intervals):
		candidate_tests = []
		for candidate in candidate_intervals:
			channel = candidate['channel']
			b = candidate['interval'][0]
			e = candidate['interval'][1]
			dist_fun = self.dist_func_sel
			threshold = random.uniform(0,2)

			ref_idx = np.random.randint(0, X.shape[0])
			cols = [col for col in X.columns if col.startswith(channel)]
			x_barr = X.iloc[ref_idx][cols].apply(lambda s: s[b:e]).values

			candidate_test = {
				'x_barr': x_barr,
				'channel': channel,
				'b': b,
				'e': e,
				'dist_fun': dist_fun,
				'threshold': threshold
			}
			candidate_tests.append(candidate_test)
		return candidate_tests

	def optimization_f(self, X, y, candidate_tests):
		entropy_test = 1000
		df_true = None
		df_false = None

		for candidate in candidate_tests:
			candidate_tests_true = []
			candidate_tests_false = []

			for i, (idx, row) in enumerate(X.iterrows()):
				col = candidate['channel']
				slice_x = row[col].iloc[candidate['b']:candidate['e']].values
				dist = np.mean(candidate['dist_fun'](slice_x, candidate['x_barr']))

				if dist <= candidate['threshold']:
					candidate_tests_true.append((row, y[i]))
				else:
					candidate_tests_false.append((row, y[i]))

			y_true = [label for _, label in candidate_tests_true]
			y_false = [label for _, label in candidate_tests_false]

			current_entropy = self.calculate_entropy(y_true, y_false)
			if current_entropy < entropy_test:
				best_test_candidate = candidate
				entropy_test = current_entropy
				df_true = candidate_tests_true
				df_false = candidate_tests_false

		return best_test_candidate, df_true, df_false

	def classification_f(self, X, y):
		classes, counts = np.unique(y, return_counts=True)
		distr = {cls: cnt/len(y) for cls, cnt in zip(classes, counts)}
		return distr

	def stopping_f(self, path, X, y):
		"""
		Stop quando l'entropia è zero (cioè tutti i campioni appartengono alla stessa classe).
		"""
		if len(y) == 0:
			return True

		_, counts = np.unique(y, return_counts=True)
		return len(counts) == 1

		
	def entropy(self, labels):
		from collections import Counter
		import math
		n = len(labels)
		if n == 0:
			return 0.0
		counts = Counter(labels)
		probs = [c/n for c in counts.values()]
		return -sum(p * math.log2(p) for p in probs if p > 0)
	
	def calculate_entropy(self, y_true, y_false):
		n_true = len(y_true)
		n_false = len(y_false)
		total = n_true + n_false
		H_true = self.entropy(y_true)
		H_false = self.entropy(y_false)
		H_total = (n_true/total) * H_true + (n_false/total) * H_false
		return H_total
	
	def get_distance_functions(self):
		return [euclidean_distance, manhattan_distance, cosine_distance]
	
	# Post-pruning
	def post_prune(self, validation_X, validation_y):
		"""Perform post-pruning on the tree using a validation set"""
		def prune_node(node, X_val, y_val):
			if isinstance(node, LeafNode):
				return node

			# Se non ci sono dati di validazione, crea foglia con la classe più comune
			if len(X_val) == 0:
				return LeafNode({self.most_common_class: 1.0})

			# Split validation set by node threshold
			X_left_rows, y_left_rows = [], []
			X_right_rows, y_right_rows = [], []

			for i in range(len(X_val)):
				row = X_val.iloc[i]
				slice_x = row[node.channel].iloc[node.begin_idx:node.end_idx].values
				dist = np.mean(node.distance_f_node(slice_x, node.x_barr))
				if dist <= node.threshold:
					X_left_rows.append(row)
					y_left_rows.append(y_val[i])
				else:
					X_right_rows.append(row)
					y_right_rows.append(y_val[i])

			# Recursively prune children
			if node.node_sx is not None:
				node.node_sx = prune_node(node.node_sx,
										  pd.DataFrame(X_left_rows, columns=X_val.columns),
										  y_left_rows)
			if node.node_dx is not None:
				node.node_dx = prune_node(node.node_dx,
										  pd.DataFrame(X_right_rows, columns=X_val.columns),
										  y_right_rows)

			# Evaluate replacing node with a leaf
			leaf_class = self.classification_f(X_val, y_val)
			leaf_node = LeafNode(leaf_class)

			correct_leaf = sum([1 for i in range(len(X_val))
								if y_val[i] == self.predict_sample_leaf(leaf_node, X_val.iloc[i])])
			correct_subtree = sum([1 for i in range(len(X_val))
								   if y_val[i] == self.predict_sample(node, X_val.iloc[i])])

			# Replace node with leaf if accuracy does not decrease
			min_samples_prune = 5
			improvement_threshold = 0.01
			if len(X_val) >= min_samples_prune and (correct_leaf - correct_subtree) >= improvement_threshold:
				return leaf_node
			else:
				return node

		self.root = prune_node(self.root, validation_X, validation_y)

	def predict_sample_leaf(self, leaf_node, x):
		"""Return predicted class from leaf (most probable)"""
		if leaf_node.classification_f is None:
			return None
		return max(leaf_node.classification_f, key=leaf_node.classification_f.get)

	def predict_sample(self, node, x):
		"""Predict label for a single sample"""
		while not isinstance(node, LeafNode):
			slice_x = x[node.channel].iloc[node.begin_idx:node.end_idx].values
			dist = np.mean(node.distance_f_node(slice_x, node.x_barr))
			node = node.node_sx if dist <= node.threshold else node.node_dx
		return self.predict_sample_leaf(node, x)
	
	# Get path from tree
	def get_leaf_node(self, tree, x):
		"""Get the leaf node where a sample ends up"""
		if tree.root is None:
			return None

		node = tree.root
		while not isinstance(node, LeafNode):
			# print(len(x))
			slice_x = x[node.begin_idx:node.end_idx]
			# slice_x = x[node.channel][node.begin_idx:node.end_idx]
			dist = np.mean(node.distance_f_node(slice_x, node.x_barr))
			if dist <= node.threshold:
				node = node.node_sx  # ramo True
			else:
				node = node.node_dx  # ramo False

			if node is None:
				return None

		return node
	
	def get_path_depth(self, tree, x):
		"""Get the path depth for a single sample"""
		if self.root is None:
			return 0

		node = self.root
		depth = 0
		while not isinstance(node, LeafNode):
			# print(len(x))
			slice_x = x[node.begin_idx:node.end_idx]
			# slice_x = x[node.channel][node.begin_idx:node.end_idx]
			dist = np.mean(node.distance_f_node(slice_x, node.x_barr))
			if dist <= node.threshold:
				node = node.node_sx  # ramo True
			else:
				node = node.node_dx  # ramo False
			if node is None:
				return None
			depth += 1
		return depth
	
	def get_shared_path_depth(self, tree, x1 , x2):
		"""Get the depth of shared path between two samples"""
		if tree.root is None:
			return 0

		node = tree.root
		depth = 0
		while not isinstance(node, LeafNode):
				slice_x1 = x1[node.begin_idx:node.end_idx]
				dist_x1 = np.mean(node.distance_f_node(slice_x1, node.x_barr))
				
				slice_x2 = x2[node.begin_idx:node.end_idx]
				dist_x2 = np.mean(node.distance_f_node(slice_x2, node.x_barr))
				
				if dist_x1 <= node.threshold:
					result1 = True
				else:
					result1 = False
				
				if dist_x2 <= node.threshold:
					result2 = True
				else:
					result2 = False
					
				if result1 != result2:
					break  # Paths diverge

				depth += 1
				node = node.node_sx if result1 else node.node_dx

				if node is None:
					break

		return depth, node

	def get_max_path_depth(self, tree, x1, x2):
		"""Get the maximum path depth for two samples"""
		depth1 = self.get_path_depth(tree, x1)
		depth2 = self.get_path_depth(tree, x2)
		return max(depth1, depth2)
	
	def go_to_leaf_from_shared_path(self, node, x):
		test_list_x = [] # per ogni test tengo del nodo e il risultato
		while not isinstance(node, LeafNode):
			slice_x = x[node.begin_idx:node.end_idx]
			dist_x = np.mean(node.distance_f_node(slice_x, node.x_barr))
			
			if dist_x <= node.threshold:
				result = True
			else:
				result = False
			test_list_x.append((node, result))
			node = node.node_sx if result else node.node_dx
			if node is None:
				break
		return test_list_x

#### **Random Forest**

In [ ]:
class RandomForest():
	def __init__(self, n_trees, X_train, y_train, X_val, y_val, X_test, y_test, distance_function):
		self.number_of_trees = n_trees
		self.X_train = X_train
		self.y_train = y_train
		self.X_val = X_val
		self.y_val = y_val
		self.X_test = X_test
		self.y_test = y_test
		
		# solo per conformal classifier
		self.X_cal = X_val
		self.y_cal = y_val
		
		self.dist_fun_for_tree = distance_function
		self.trees = self.create_trees()

		# dizionari per salvare pesi e track record
		self.weights_random_forest = None
		self.track_records_random_forest = None

	def create_trees(self):
		trees = {}
		for i in range(self.number_of_trees):
			tree = PromptTree(self.X_train, self.y_train, self.dist_fun_for_tree)
			trees[i] = tree
		return trees 
	
	def predict_only(self, X):
		"""
		Restituisce:
		  - y_probs_final: lista di dizionari con la distribuzione finale combinata di tutti gli alberi
		  - y_probs_per_tree: lista di liste di dizionari, ciascuna sotto-lista contiene le distribuzioni di probabilità
			di ciascun albero per un campione
		"""
		y_probs_final = []
		y_probs_per_tree = []  # lista di liste

		for _, x in X.iterrows():
			class_scores = {}
			probs_per_tree = []

			for tree in self.trees.values():
				pred = predict(tree, x, default_class=0)
				# Ogni albero restituisce solo 1 probabilità 1 per la predetta e 0 per le altre
				probs_tree = {pred: 1.0}  # dizionario con predizione dell'albero
				probs_per_tree.append(probs_tree)

				# accumula punteggio per combinazione finale
				class_scores[pred] = class_scores.get(pred, 0) + 1

			# Normalizzazione in probabilità finale
			total_score = sum(class_scores.values())
			probs_final = {cls: score / total_score for cls, score in class_scores.items()}

			y_probs_final.append(probs_final)
			y_probs_per_tree.append(probs_per_tree)

		return y_probs_final, y_probs_per_tree


	def majority_voting(self, X, return_probs=False):
		y_final_pred = []
		y_probs = [] 
		for _, x in X.iterrows():
			class_votes = {}
			for tree in self.trees.values():
				pred = predict(tree, x, default_class=0)
				class_votes[pred] = class_votes.get(pred, 0) + 1
			
			# predizione finale
			final_cls = max(class_votes.items(), key=lambda kv: kv[1])[0]
			y_final_pred.append(final_cls)
			
			if return_probs:
				total_votes = sum(class_votes.values())
				probs = {cls: v/total_votes for cls, v in class_votes.items()}
				y_probs.append(probs)
		
		accuracy_test = sum(p == t for p, t in zip(y_final_pred, self.y_test)) / len(self.y_test)
		if return_probs:
			return round(accuracy_test, 3), y_final_pred, y_probs
		else:
			return round(accuracy_test, 3)

	def weighted_voting(self, X_val, y_val, X_test, y_test, return_probs=False):
		# Calcolo le accuracy in validation per ogni tree
		accuracies_val = {}
		for i, tree in self.trees.items():
			y_pred = [predict(tree, x, default_class=0) for _, x in X_val.iterrows()]
			acc = sum(y_pred_i == y_true_i for y_pred_i, y_true_i in zip(y_pred, y_val)) / len(y_val)
			accuracies_val[i] = acc

		# Normalizzo
		total = sum(accuracies_val.values())
		if total == 0:
			weights = {i: 1/len(self.trees) for i in self.trees}
		else:
			weights = {i: acc/total for i, acc in accuracies_val.items()}

		# Salvo i pesi nella variabile di istanza
		self.weights_random_forest = weights

		# Final prediction pesata
		y_final_pred = []
		y_probs = []
		for _, x in X_test.iterrows():
			class_scores = {}
			for i, tree in self.trees.items():
				pred = predict(tree, x, default_class=0)
				class_scores[pred] = class_scores.get(pred, 0) + weights[i]

			final_cls = max(class_scores.items(), key=lambda kv: kv[1])[0]
			y_final_pred.append(final_cls)

			if return_probs:
				total_score = sum(class_scores.values())
				probs = {cls: s/total_score for cls, s in class_scores.items()}
				y_probs.append(probs)

		accuracy_test = sum(p == t for p, t in zip(y_final_pred, y_test)) / len(y_test)
		if return_probs:
			return round(accuracy_test, 3), y_final_pred, y_probs
		else:
			return round(accuracy_test, 3)

	def track_record_voting(self, X_val, y_val, X_test, y_test, return_probs=True):
		# Ogni tree accumula un record di performance su un validation set
		accuracies_val_for_tree = {}
		for i, tree in self.trees.items():
			class_correct = {}
			class_total = {}

			for idx, (_, x) in enumerate(X_val.iterrows()):
				y_pred_i = predict(tree, x, default_class=0)
				y_true_i = y_val[idx]

				# incremento i conteggi per la classe vera
				class_total[y_true_i] = class_total.get(y_true_i, 0) + 1
				if y_pred_i == y_true_i:
					class_correct[y_true_i] = class_correct.get(y_true_i, 0) + 1

			# salvo accuracies per classe
			accuracies_val_for_tree[i] = {
				cls: class_correct.get(cls, 0) / total for cls, total in class_total.items()
			}

		# Salvo il track record nella variabile di istanza
		self.track_records_random_forest = accuracies_val_for_tree

		# Final prediction
		y_final_pred = []
		y_probs = []
		for _, x in X_test.iterrows():
			class_scores = {}
			for i, tree in self.trees.items():
				pred = predict(tree, x, default_class=0)
				weight = accuracies_val_for_tree[i].get(pred, 0)
				class_scores[pred] = class_scores.get(pred, 0) + weight

			final_cls = max(class_scores.items(), key=lambda kv: kv[1])[0]
			y_final_pred.append(final_cls)

			if return_probs:
				total_score = sum(class_scores.values())
				if total_score == 0:
					# Distribuisci uniformemente le probabilità tra tutte le classi
					n_classes = len(class_scores)
					probs = {cls: 1/n_classes for cls in class_scores}
				else:
					probs = {cls: s/total_score for cls, s in class_scores.items()}
				y_probs.append(probs)


		accuracy_test = sum(p == t for p, t in zip(y_final_pred, y_test)) / len(y_test)
		if return_probs:
			return round(accuracy_test, 3), y_final_pred, y_probs
		else:
			return round(accuracy_test, 3)

#### **Conformal classifier**

In [ ]:
class ConformalClassifier:
	def __init__(self, classifier, voting="majority", alpha=0.1):
		"""
		classifier: istanza della tua RandomForest
		voting: "majority", "weighted", "track_record"
		"""
		self.classifier = classifier
		self.voting = voting
		self.alpha = alpha

	def get_pred_probs_calibration(self):
		"""
		Ritorna le predizioni probabilistiche sul set di calibrazione
		secondo il meccanismo di voting scelto.
		"""
		X_cal = self.classifier.X_cal 
		y_cal = self.classifier.y_cal
		y_probs = []
		
		y_cal_preds_probs, y_cal_preds_probs_per_tree = self.classifier.predict_only(X_cal)

		if self.voting == "majority":
			probs = y_cal_preds_probs
		elif self.voting == "weighted":
			# best_pred_for_each_sample = []
			final_probs = []
			for pred_prob_per_tree in y_cal_preds_probs_per_tree: # itero sulle predizioni per ogni sample
				combined_scores = {}
				for idx_of_tree, tree_probs in enumerate(pred_prob_per_tree):
					# print(str(tree_probs) + " ---- " + str(idx_of_tree))
					for label, prob in tree_probs.items():
					# print(self.classifier.weights_random_forest)
						tree_weight = self.classifier.weights_random_forest[idx_of_tree] # peso dell'albero
						combined_scores[label] = combined_scores.get(label, 0) + prob * tree_weight
				total_score = sum(combined_scores.values())
				if total_score > 0:
					normalized_scores = {cls: score / total_score for cls, score in combined_scores.items()}
				else:
					normalized_scores = {cls: 0 for cls in combined_scores}

				# tengo solo la predizione con prob massima
				# best_pred = max(normalized_scores.items(), key=lambda kv: kv[1])
				# best_pred = {best_pred[0]: best_pred[1]}
				# best_pred_for_each_sample.append(best_pred)
				final_probs.append(normalized_scores)
			probs = final_probs
		elif self.voting == "track_record":
			# best_pred_for_each_sample = []
			final_probs = []
			for pred_prob_per_tree in y_cal_preds_probs_per_tree:  # itero sulle predizioni per ogni sample
				combined_scores = {}
				for idx_of_tree, tree_probs in enumerate(pred_prob_per_tree):  # per ogni albero
					for label, prob in tree_probs.items():
						# prendo il peso dal track record dell'albero per quella classe
						tree_record = self.classifier.track_records_random_forest[idx_of_tree]
						class_weight = tree_record.get(label, 0)  # prendo il peso relativo alla classe specifica, se quella classe non era mai stata vista → 0
						combined_scores[label] = combined_scores.get(label, 0) + prob * class_weight
				
				# normalizzazione dei punteggi per ottenere distribuzione di probabilità
				total_score = sum(combined_scores.values())
				if total_score > 0:
					normalized_scores = {cls: score / total_score for cls, score in combined_scores.items()}
				else:
					normalized_scores = {cls: 0 for cls in combined_scores}
			
				# tengo solo la predizione con prob massima
				# best_pred = max(normalized_scores.items(), key=lambda kv: kv[1])
				# best_pred = {best_pred[0]: best_pred[1]}
				# best_pred_for_each_sample.append(best_pred)
				final_probs.append(normalized_scores)
			probs = final_probs
		else:
			raise ValueError("Voting must be 'majority', 'weighted' or 'track_record'")
		# print(probs)
		# print(len(probs))
		return probs, y_cal

	def conformal_predict(self):
		"""
		Calcola le predizioni conformi sul test set.
		Ritorna una lista di liste: classi conformi per ogni sample.
		"""
		# Calcolo predizioni probabilistiche sul set di calibrazione
		pred_probs_cal, y_cal = self.get_pred_probs_calibration()

		# Calcolo i non-conformity scores
		nc_scores = []
		for i, probs in enumerate(pred_probs_cal):
			true_class = y_cal[i] # questa è la vera label, la vera classe
			# print(true_class, i, probs)
			nc_score = 1 - probs.get(true_class, 0) # capisco quanto la mia predizione della classe vera si discosta dalla predizione della classe vera del dataset
			nc_scores.append(nc_score)
		# print("\nNC scores", nc_scores)
		# print(len(nc_scores))

		# Calcolo il threshold di conformità
		threshold = np.quantile(nc_scores, 1 - self.alpha)

		# Predizione su X_test
		y_conformal = []
		# for _, x in X_test.iterrows():
		# Ottengo probabilità predette dal modello
		if self.voting == "majority":
			_, _, probs_test = self.classifier.majority_voting(self.classifier.X_test, return_probs=True)
		elif self.voting == "weighted":
			_, _, probs_test = self.classifier.weighted_voting(self.classifier.X_val, self.classifier.y_val, self.classifier.X_test, self.classifier.y_test, return_probs=True)
		elif self.voting == "track_record":
			_, _, probs_test = self.classifier.track_record_voting(self.classifier.X_val, self.classifier.y_val, self.classifier.X_test, self.classifier.y_test, return_probs=True)

		# Conformal set per questo sample
		# probs_test è lista di dict, un dict per sample contenente tutte le label e le relative probabilità
		# print(probs_test)
		for i, sample_probs in enumerate(probs_test):  # iteriamo su ogni sample
			conformal_set = [
				cls
				for cls, p in sample_probs.items() # prendo la classe/label e la probabilità relativa
				if (1 - p) <= threshold # memorizzo la classe nel conformal set solo se > threshold
			]

			# Evita duplicati (non necessario se sample_probs è già un dict)
			y_conformal.append(conformal_set)

		return y_conformal
		
	def calibrate(self, alpha=0.1):
		y_conformal = self.conformal_predict() # miscalibration=coverage desiderata−coverage osservata
		
		# Coverage desiderata
		confidence = 1 - alpha
		
		# Efficiency = percentuale di sample con solo 1 classe conforme
		efficiency = sum(len(c) == 1 for c in y_conformal) / len(y_conformal)
		
		# Coverage osservata: quante volte la vera classe è nell'insieme conforme
		coverage_observed = sum(
			true_label in conf_classes
			for true_label, conf_classes in zip(self.classifier.y_cal, y_conformal)
		) / len(y_conformal)
		
		# Miscalibration
		miscalibration = confidence - coverage_observed
		
		return confidence, efficiency, miscalibration

#### **Unsupervised learning**

In [ ]:
# distance functions
def breiman_distance(forest, x1, x2):
	"""Breiman distance: fraction of trees where samples end up in different leaves"""
	if not forest.trees:
		return 1.0

	different_leaves = 0
	valid_trees = 0
	for idx_tree, tree in forest.trees.items(): # per ogni albero
		# Get leaf nodes for both samples, that are a list of numbers
		# print("X1:", x1)
		# print("X2:", x2)
		leaf1 = tree.get_leaf_node(tree, x1)
		leaf2 = tree.get_leaf_node(tree, x2)

		if leaf1 != leaf2:
			different_leaves += 1
		valid_trees += 1
	return different_leaves / valid_trees if valid_trees > 0 else 1.0

def zhu_distance(forest, x1, x2):
	"""Zhu distance: based on shared path length in trees"""
	if not forest.trees:
		return 1.0

	total_shared_depth = 0
	total_max_depth = 0

	for idx_tree, tree in forest.trees.items(): # per ogni albero
		shared_depth, _ = tree.get_shared_path_depth(tree, x1, x2)
		max_depth = tree.get_max_path_depth(tree, x1, x2)

		total_shared_depth += shared_depth
		total_max_depth += max_depth

	if total_max_depth == 0:
		return 1.0

	return 1.0 - (total_shared_depth / total_max_depth)

def ratio_rf_distance(forest, x1, x2):
	"""RatioRF distance: ratio-based distance measure"""
	if not forest.trees:
		return 1.0

	shared_results_for_tree = {}
	for idx_tree, tree in forest.trees.items(): # per ogni albero
		shared_depth, node = tree.get_shared_path_depth(tree, x1, x2) # prendo il path condiviso nell'albero
		max_depth = tree.get_max_path_depth(tree, x1, x2) # to normalize value when shared_results_x1 or shared_results_x2 are zero 
		
		# print('Shared depth:', shared_depth)
		# print('Max depth:', max_depth)
		# adesso sia per x1 che x2 colleziono i testi mentre scendono nell'albero dall'ultimo nodo condiviso
		
		test_list_x1 = tree.go_to_leaf_from_shared_path(node, x1) # ritornano una tupla (node, result(True/False))
		test_list_x2 = tree.go_to_leaf_from_shared_path(node, x2)

		# itero attraverso il dict, faccio il test sull'altro sample e confronto i risultati
		diff_results_x1 = 0
		for node_and_result in test_list_x1:
			node = node_and_result[0]
			result_x1 = node_and_result[1]
			slice_x2 = x2[node.begin_idx:node.end_idx]
			dist_x2 = np.mean(node.distance_f_node(slice_x2, node.x_barr))
			
			if dist_x2 <= node.threshold:
				result_x2 = True
			else:
				result_x2 = False
			
			# confronti i risultati
			if result_x1 != result_x2: 
				diff_results_x1 += 1 # conto le discrepanze
		
		diff_results_x2 = 0
		for node_and_result in test_list_x2:
			node = node_and_result[0]
			result_x2 = node_and_result[1]
			slice_x1 = x1[node.begin_idx:node.end_idx]
			dist_x1 = np.mean(node.distance_f_node(slice_x1, node.x_barr))
			
			if dist_x1 <= node.threshold:
				result_x1 = True
			else:
				result_x1 = False
			
			# confronti i risultati
			if result_x1 != result_x2:
				diff_results_x2 += 1
		
		############
		
		# based on tversky formula, alpha and beta are the 2 weights to give to the two shared results
		alpha = 0.5 # give same importance
		beta = 0.5 
		
		if diff_results_x1 == 0 and diff_results_x2 == 0:
			shared_results_for_tree[idx_tree] = shared_depth / max_depth # normalizzo lo shared path rispetto la distanza totale
		else:
			shared_results_for_tree[idx_tree] = shared_depth / (shared_depth + alpha * diff_results_x1 + beta * diff_results_x2)
	
	similarity = 0
	# print("Shared results for tree:", shared_results_for_tree)
	for _, shared_result in shared_results_for_tree.items():
		similarity += shared_result
	return 1.0 - similarity / forest.number_of_trees

# distance function for cluster similarity
def euclidean_distance_clustering(x1, x2):
	"""Calcola la distanza Euclidea tra due sample flattenati§"""
	x1 = np.array(x1)
	x2 = np.array(x2)

	return np.linalg.norm(x1 - x2)

##### **Isolation forest**

In [ ]:
class IsolationForest:

	def __init__(self, rf, n_clusters, distance_type):
		self.forest = rf
		self.distance_functions = self.get_distance_functions()
		self.n_clusters = n_clusters
		self.distance_type = distance_type
		self.euclidean_distance = euclidean_distance_clustering

	def get_distance_functions(self):
		return {
			'breiman': breiman_distance,
			'zhu': zhu_distance,
			'ratio_rf': ratio_rf_distance
		}
	
	def perform_cluster(self, X):
		"""Perform hierarchical clustering based on forest distances"""
		n_samples = len(X)

		# Calculate distance matrix
		distance_matrix = np.zeros((n_samples, n_samples))
		
		distance_func = self.distance_functions[self.distance_type]
		for i in range(n_samples):
			for j in range(i + 1, n_samples):
				# print("Xi:", len(X[i]))
				dist = distance_func(self.forest, X[i], X[j])
				distance_matrix[i, j] = dist
				distance_matrix[j, i] = dist
				
		# print(distance_matrix)
		# Dopo aver calcolato la distance_matrix
		plt.imshow(distance_matrix, cmap="viridis")
		plt.colorbar()
		plt.title("Matrice delle distanze")
		plt.show()

		# Perform hierarchical clustering
		condensed_dist = squareform(distance_matrix) # squareform trasforma la matrice quadrata in un array condensato, richiesto dalla funzione linkage.
		linkage_matrix = linkage(condensed_dist, method='average') # linkage costruisce un albero gerarchico (dendrogramma) usando il metodo di collegamento “average”
		cluster_labels = fcluster(linkage_matrix, self.n_clusters, criterion='maxclust') # fcluster taglia l’albero in n_clusters cluster, restituendo le etichette dei cluster.

		return cluster_labels - 1  # parto dal cluster 0, invece che dall'1
	
	def purity(self, true_labels, cluster_labels):
		"""Calculate cluster purity"""
		n_samples = len(true_labels)
		purity = 0

		for cluster_id in set(cluster_labels):
			cluster_mask = np.array(cluster_labels) == cluster_id
			cluster_true_labels = np.array(true_labels)[cluster_mask]

			if len(cluster_true_labels) > 0:
				most_common_class = Counter(cluster_true_labels).most_common(1)[0][1]
				purity += most_common_class

		return purity / n_samples

	def entropy(self, true_labels, cluster_labels):
		"""Calculate cluster entropy"""
		total_entropy = 0
		n_samples = len(true_labels)

		for cluster_id in set(cluster_labels):
			cluster_mask = np.array(cluster_labels) == cluster_id
			cluster_true_labels = np.array(true_labels)[cluster_mask]
			cluster_size = len(cluster_true_labels)

			if cluster_size > 0:
				class_counts = Counter(cluster_true_labels)
				cluster_entropy = 0

				for count in class_counts.values():
					if count > 0:
						prob = count / cluster_size
						cluster_entropy -= prob * np.log2(prob)

				total_entropy += (cluster_size / n_samples) * cluster_entropy

		return total_entropy

	def adjusted_rand_index(self, true_labels, cluster_labels):
		"""Calculate Adjusted Rand Index between two clusterings
		valuta quante coppie di punti sono assegnate allo stesso cluster 
		in entrambe le partizioni rispetto a quanto ci si aspetterebbe per caso"""
		
		# ARI = 1 > le due partizioni sono identiche, ARI ≈ 0 -> la somiglianza è quella che ci si aspetta per caso.
		return adjusted_rand_score(true_labels, cluster_labels) # funzione di sklearn

	def intra_cluster_distance(self, X, cluster_labels):
		"""Average intra-cluster distance, Euclidian distance"""
		total_distance = 0
		total_pairs = 0

		for cluster_id in set(cluster_labels): # insieme di cluster unici, tipo {0, 1, 2, 3, 4}
			cluster_indices = [i for i, label in enumerate(cluster_labels) # per ogni cluster, tiro fuori i sample appartenenti a quel cluster
							 if label == cluster_id]
			# print("cluster indices:", cluster_indices)
			for i in range(len(cluster_indices)):
				for j in range(i + 1, len(cluster_indices)):
					idx1, idx2 = cluster_indices[i], cluster_indices[j]
					dist = self.euclidean_distance(X[idx1], X[idx2])
					total_distance += dist
					total_pairs += 1

		return total_distance / total_pairs if total_pairs > 0 else 0

	def inter_cluster_distance(self, X, cluster_labels):
		"""Average inter-cluster distance, Euclidian distance"""
		total_distance = 0
		total_pairs = 0

		unique_clusters = list(set(cluster_labels))

		for i in range(len(unique_clusters)):
			for j in range(i + 1, len(unique_clusters)):
				cluster1_indices = [k for k, label in enumerate(cluster_labels)
								  if label == unique_clusters[i]]
				cluster2_indices = [k for k, label in enumerate(cluster_labels)
								  if label == unique_clusters[j]]

				for idx1 in cluster1_indices:
					for idx2 in cluster2_indices:
						dist = self.euclidean_distance(X[idx1], X[idx2])
						total_distance += dist
						total_pairs += 1

		return total_distance / total_pairs if total_pairs > 0 else 0

### **Main**

In [ ]:
# ======================
# 1. DATA LOADING
# ======================
print("\n======================")
print("1. DATA LOADING")
print("======================")

# Carica dataset (scegli univariato o multivariato)
# X_train, y_train = load_from_tsfile_to_dataframe("C:/Users/Simone/Desktop/UNIVERSITA/MAGISTRALE/BIOMEDICAL DECISION SUPPORT SYSTEM/Univariate_ts/Car/Car_TRAIN.ts") 
# X_test, y_test = load_from_tsfile_to_dataframe("C:/Users/Simone/Desktop/UNIVERSITA/MAGISTRALE/BIOMEDICAL DECISION SUPPORT SYSTEM/Univariate_ts/Car/Car_TEST.ts")

# X_train, y_train = load_from_tsfile_to_dataframe("C:/Users/Simone/Desktop/UNIVERSITA/MAGISTRALE/BIOMEDICAL DECISION SUPPORT SYSTEM/Multivariate_ts/HandMovementDirection/HandMovementDirection_TRAIN.ts") 
# X_test, y_test = load_from_tsfile_to_dataframe("C:/Users/Simone/Desktop/UNIVERSITA/MAGISTRALE/BIOMEDICAL DECISION SUPPORT SYSTEM/Multivariate_ts/HandMovementDirection/HandMovementDirection_TEST.ts") 

X_train, y_train = load_from_tsfile_to_dataframe("C:/Users/Simone/Desktop/UNIVERSITA/MAGISTRALE/BIOMEDICAL DECISION SUPPORT SYSTEM/Univariate_ts/Beef/Beef_TRAIN.ts") 
X_test, y_test = load_from_tsfile_to_dataframe("C:/Users/Simone/Desktop/UNIVERSITA/MAGISTRALE/BIOMEDICAL DECISION SUPPORT SYSTEM/Univariate_ts/Beef/Beef_TEST.ts")

# X_train, y_train = load_from_tsfile_to_dataframe("C:/Users/Simone/Desktop/UNIVERSITA/MAGISTRALE/BIOMEDICAL DECISION SUPPORT SYSTEM//Multivariate_ts/AtrialFibrillation/AtrialFibrillation_TRAIN.ts") 
# X_test, y_test = load_from_tsfile_to_dataframe("C:/Users/Simone/Desktop/UNIVERSITA/MAGISTRALE/BIOMEDICAL DECISION SUPPORT SYSTEM/Multivariate_ts/AtrialFibrillation/AtrialFibrillation_TEST.ts") 

# Split: train, validation, calibration
X_train, X_val, y_train, y_val = train_test_split(
	X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)
X_train, X_cal, y_train, y_cal = train_test_split(
	X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

# Encoding labels
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_val   = le.transform(y_val)
y_cal   = le.transform(y_cal)
y_test  = le.transform(y_test)

print("X_train:", len(X_train))
print("y_train:", len(y_train))
print("X_val:", len(X_val))
print("y_val:", len(y_val))
print("X_cal:", len(X_cal))
print("y_cal:", len(y_cal))
print("X_test:", len(X_test))
print("y_test:", len(y_test))


# ======================
# 2. DEBUG CHECK
# ======================
print("\n======================")
print("2. DEBUG CHECK")
print("======================")

for i in range(2):
	print(f"Trace {i}:")
	for col in X_train.columns:
		serie = X_train.iloc[i][col]
		print(f"  Column {col}: {serie.head(10).tolist()}")
	print("Label:", y_train[i])
	print("----")

print("\nShape of train set:", X_train.shape)


# ======================
# 3. PROMPT TREE
# ======================
print("\n======================")
print("3. PROMPT TREE")
print("======================")

distance_function = euclidean_distance # cosine_distance, euclidean_distance, manhattan_distance
prompt_tree = PromptTree(X_train, y_train, distance_function)

# Funzione per stampare l’albero
def print_tree(node, prefix=""):
	if isinstance(node, LeafNode):
		print(f"{prefix}└── Leaf: class={node.classification_f}")
	elif isinstance(node, Node):
		print(f"{prefix}└── Node: channel={node.channel}, "
			  f"begin={node.begin_idx}, end={node.end_idx}, "
			  f"threshold={node.threshold}")
		if node.node_sx is not None:
			print(f"{prefix}    Left:")
			print_tree(node.node_sx, prefix + "        ")
		if node.node_dx is not None:
			print(f"{prefix}    Right:")
			print_tree(node.node_dx, prefix + "        ")

# Funzione di predizione
def predict(tree, x, default_class=None):
	node = tree.root
	while isinstance(node, Node):
		slice_x = x[node.channel][node.begin_idx:node.end_idx]
		dist = np.mean(node.distance_f_node(slice_x, node.x_barr))
		node = node.node_sx if dist <= node.threshold else node.node_dx

	if not node.classification_f:
		return default_class
	return max(node.classification_f.items(), key=lambda kv: kv[1])[0]

# Accuracy su test con PromptTree singolo
y_pred = [predict(prompt_tree, x, default_class=0) for _, x in X_test.iterrows()]
acc = round(np.mean([yp == yt for yp, yt in zip(y_pred, y_test)]), 3)
print("\nClassification Report:")
print(classification_report(y_test, y_pred, digits=3))


# ======================
# 4. RANDOM FOREST
# ======================
print("\n======================")
print("4. RANDOM FOREST")
print("======================")

n_trees = 5
distance_function = cosine_distance
random_forest = RandomForest(n_trees, X_train, y_train, X_val, y_val, X_test, y_test, distance_function)

return_probs = True

# Confusion matrix helper
def plot_confusion_matrix(y_true, y_pred, title="Confusion Matrix"):
	cm = confusion_matrix(y_true, y_pred)
	plt.figure(figsize=(6,5))
	sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
	plt.xlabel("Predicted")
	plt.ylabel("True")
	plt.title(title)
	plt.show()

# Majority voting
print("\n--- Majority Voting ---")
acc_maj, y_pred_maj, y_probs_maj = random_forest.majority_voting(random_forest.X_test, return_probs=return_probs)
print("Accuracy:", acc_maj)
print("Classification Report:\n", classification_report(random_forest.y_test, y_pred_maj))
plot_confusion_matrix(random_forest.y_test, y_pred_maj, "Majority Voting")

# Weighted voting
print("\n--- Weighted Voting ---")
acc_wei, y_pred_wei, y_probs_wei = random_forest.weighted_voting(
	random_forest.X_val, random_forest.y_val,
	random_forest.X_test, random_forest.y_test,
	return_probs=return_probs
)
print("Accuracy:", acc_wei)
print("Classification Report:\n", classification_report(random_forest.y_test, y_pred_wei))
plot_confusion_matrix(random_forest.y_test, y_pred_wei, "Weighted Voting")

# Track record voting
print("\n--- Track Record Voting ---")
acc_tr, y_pred_tr, y_probs_tr = random_forest.track_record_voting(
	random_forest.X_val, random_forest.y_val,
	random_forest.X_test, random_forest.y_test,
	return_probs=return_probs
)
print("Accuracy:", acc_tr)
print("Classification Report:\n", classification_report(random_forest.y_test, y_pred_tr))
plot_confusion_matrix(random_forest.y_test, y_pred_tr, "Track Record Voting")

print("\nRF weights for each tree:", random_forest.weights_random_forest)
print("RF track records for each tree:", random_forest.track_records_random_forest)


# ======================
# 5. CONFORMAL CLASSIFIER
# ======================
print("\n======================")
print("5. CONFORMAL CLASSIFIER")
print("======================")

voting = 'weighted'
alpha = 0.1

conformal_classifier = ConformalClassifier(random_forest, voting, alpha)
confidence, efficiency, miscalibration = conformal_classifier.calibrate()
print(f"Confidence: {confidence*100:.1f}%")
print(f"Efficiency: {efficiency*100:.1f}%")
print(f"Miscalibration: {miscalibration*100:.1f}%")


# ======================
# 6. ISOLATION FOREST (CLUSTERING)
# ======================
print("\n======================")
print("6. ISOLATION FOREST (CLUSTERING)")
print("======================")

n_clusters = 5
distance_type = 'ratio_rf'
isolation_forest = IsolationForest(random_forest, n_clusters, distance_type)

# Flatten delle serie (concatenazione per riga)
X_train_flat = np.array([np.concatenate([chan for chan in row]) for _, row in X_train.iterrows()])

# Clustering
cluster_labels = isolation_forest.perform_cluster(X_train_flat)
print("Cluster labels:", cluster_labels)
print("Num labels == num samples:", len(cluster_labels) == len(X_train_flat))

# Metriche clustering
intra_cluster_dist = isolation_forest.intra_cluster_distance(X_train_flat, cluster_labels)
inter_cluster_dist = isolation_forest.inter_cluster_distance(X_train_flat, cluster_labels)
purity = isolation_forest.purity(y_train, cluster_labels)
entropy = isolation_forest.entropy(y_train, cluster_labels)
ari = isolation_forest.adjusted_rand_index(y_train, cluster_labels)

print("Intra cluster distance:", intra_cluster_dist)
print("Inter cluster distance:", inter_cluster_dist)
print("Purity:", purity)
print("Entropy:", entropy)
print("ARI:", ari)

In [ ]:
# Test with all distances and more clusters

n_clusters = 10
distance_type = 'breiman'
isolation_forest = IsolationForest(random_forest, n_clusters, distance_type)

# Flatten delle serie (concatenazione per riga)
X_train_flat = np.array([np.concatenate([chan for chan in row]) for _, row in X_train.iterrows()])

distance_types = ['breiman', 'zhu', 'ratio_rf']
results = []

for dist in distance_types:
	print(f"\n--- Clustering con distanza: {dist} ---")
	isolation_forest = IsolationForest(random_forest, n_clusters, distance_type=dist)
	cluster_labels = isolation_forest.perform_cluster(X_train_flat)

	intra_cluster_dist = isolation_forest.intra_cluster_distance(X_train_flat, cluster_labels)
	inter_cluster_dist = isolation_forest.inter_cluster_distance(X_train_flat, cluster_labels)
	purity = isolation_forest.purity(y_train, cluster_labels)
	entropy = isolation_forest.entropy(y_train, cluster_labels)
	ari = isolation_forest.adjusted_rand_index(y_train, cluster_labels)

	results.append({
		"Distance": dist,
		"IntraDist": intra_cluster_dist,
		"InterDist": inter_cluster_dist,
		"Purity": purity,
		"Entropy": entropy,
		"ARI": ari
	})

# Confronto tabellare
import pandas as pd
df_results = pd.DataFrame(results)
print("\nConfronto clustering:")
print(df_results)
